In [1]:

from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")
sqlContext = SQLContext(sc)

In [2]:
import os, time
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)

In [3]:
import re
#regex = re.compile('\s*\<row\s+(\w+)=\"(.*?)\".*(\w+)=\"(\d+)\"')
#if re.match('\s*<row\s*(\w+=".*?"\s*)*/>',line):
regex = re.compile('\s*<row\s*(\w+=".*?"\s*)*/>')
valid_stack_posts_lines=sc.textFile(localpath("spark-stack-data/allPosts/"))\
.filter(lambda x: (" />" in x)&("  <row" in x)&(' Tags=\"' in x))

#valid_stack_posts_lines=sc.textFile(localpath("spark-stack-data/allPosts/"))\
#.filter(lambda x: (" />" in x)&("  <row" in x)&(' CreationDate=\"' in x))
#.filter(lambda x: regex.match(x))
#valid_stack_users_lines = sc.textFile(localpath("spark-stack-data/allUsers/"))\
#.filter(lambda x: (" />" in x)&("  <row" in x)&(' CreationDate=\"' in x)&(' Id=\"' in x))
#.filter(lambda x: regex.match(x))
#valid_stack_posts_lines_body=valid_stack_posts_lines.map(lambda x: re.search(regex,x)[0].split(" "))

In [7]:
valid_stack_posts_lines.take(1)

['  <row AnswerCount="3" Body="&lt;p&gt;I need to launch Chrome from command line with custom parameter, which&#10;contains path to some js-file. Further this path will be used in&#10;extension.&lt;/p&gt;&#10;&#10;&lt;p&gt;I browsed carefully all related documentation and clicked all nodes in&#10;Chrome debugger, but found nothing which can resemble on command line&#10;parameters. Is it possible anyway to get these parameters or it\'s need&#10;to write more complex npapi-extension? (theoretically in such npapi-&#10;extension we able to get self process through win-api, command line of&#10;self process and so on). &lt;/p&gt;&#10;" CommentCount="4" CreationDate="2012-01-04T11:10:39.993" FavoriteCount="0" Id="8725845" LastActivityDate="2014-04-09T10:20:45.013" LastEditDate="2012-01-10T09:19:01.170" LastEditorUserId="785541" OwnerUserId="1129631" PostTypeId="1" Score="4" Tags="&lt;google-chrome&gt;&lt;command-line&gt;&lt;google-chrome-extension&gt;" Title="Is there any way to get command l

In [4]:
#get tags
regex2 = re.compile('\s(Tags="(.*?)")\s')
regex3 = re.compile('\&([^\s]*?)\;')
tag_lines=valid_stack_posts_lines\
.map(lambda x: re.search(regex2,x).group(2))\
.map(lambda x: re.sub(regex3,' ',x))\
.map(lambda x: re.sub('\s+',' ',x))\
.map(lambda x: x.strip(" ")).map(lambda line: line.split(" ")).persist()#.cache()

In [ ]:
#clean up
regex1 = re.compile('\s*<row\s*(\w+=".*?"\s*)*/>')
regex2 = re.compile('.*(Body="(.*?)")\s([A-Z]\w+="\d+")')
regex3 = re.compile('\&([^\s]*?)\;')
regex4 = re.compile('http[^\s]*')
valid_stack_posts_lines=sc.textFile(localpath("spark-stats-data/allPosts/"))\
.filter(lambda x: (re.match(regex1,x))).map(lambda x: re.search(regex2,x).group(2))\
.map(lambda x: re.sub(regex3,' ',x)).map(lambda x: re.sub(regex4,' ',x))\
.map(lambda x: re.sub('\/p|\/h2| [A-Z] | [a-z] ',' ',x)).map(lambda x: re.sub('\s+',' ',x))\
.map(lambda x: x.strip(" "))

In [20]:
valid_stack_posts_lines.take(1)

['  <row AnswerCount="3" Body="&lt;p&gt;I need to launch Chrome from command line with custom parameter, which&#10;contains path to some js-file. Further this path will be used in&#10;extension.&lt;/p&gt;&#10;&#10;&lt;p&gt;I browsed carefully all related documentation and clicked all nodes in&#10;Chrome debugger, but found nothing which can resemble on command line&#10;parameters. Is it possible anyway to get these parameters or it\'s need&#10;to write more complex npapi-extension? (theoretically in such npapi-&#10;extension we able to get self process through win-api, command line of&#10;self process and so on). &lt;/p&gt;&#10;" CommentCount="4" CreationDate="2012-01-04T11:10:39.993" FavoriteCount="0" Id="8725845" LastActivityDate="2014-04-09T10:20:45.013" LastEditDate="2012-01-10T09:19:01.170" LastEditorUserId="785541" OwnerUserId="1129631" PostTypeId="1" Score="4" Tags="&lt;google-chrome&gt;&lt;command-line&gt;&lt;google-chrome-extension&gt;" Title="Is there any way to get command l

In [5]:
from pyspark.ml.feature import Word2Vec
import time
start = time.time()
WORD_RE = re.compile(r"\w+")
gutenberg = valid_stack_posts_lines.map(lambda line: (WORD_RE.findall(line), 1)).toDF(['text', 'score'])
w2v = Word2Vec(inputCol="text", outputCol="vectors", vectorSize=100, minCount=40, seed=42)
model = w2v.fit(gutenberg)
result = model.transform(gutenberg)
vectors = model.getVectors().rdd.map(lambda x: (x.word, x.vector))
print(model.findSynonyms('ggplot2', 25).rdd.take(25))
end = time.time()
print((end - start)/60)

KeyboardInterrupt: 

In [5]:
#2for tags
import time
start = time.time()
#WORD_RE = re.compile(r"\w+")
from pyspark.mllib.feature import Word2Vec
model = Word2Vec().setVectorSize(100).setSeed(42).fit(tag_lines)
syms = model.findSynonyms("ggplot2", 25)
end = time.time()
print((end - start)/60)

18.34527146021525


In [6]:
for word, cosine_distance in syms:
    print("('{}' , {}),".format(word, cosine_distance))

('lattice' , 0.8968920111656189),
('r-grid' , 0.8583241105079651),
('plotrix' , 0.85544353723526),
('ecdf' , 0.8365693092346191),
('boxplot' , 0.8297217488288879),
('line-plot' , 0.8284202218055725),
('quantile' , 0.8263535499572754),
('levelplot' , 0.818580687046051),
('plotmath' , 0.8142644762992859),
('tapply' , 0.8135511875152588),
('density-plot' , 0.8126519918441772),
('performanceanalytics' , 0.8111131191253662),
('categorical-data' , 0.8068597316741943),
('gridextra' , 0.8055187463760376),
('mgcv' , 0.802337646484375),
('kriging' , 0.8017113208770752),
('gam' , 0.8005327582359314),
('loess' , 0.7993765473365784),
('gtable' , 0.7969805598258972),
('r-factor' , 0.7967932820320129),
('rgl' , 0.7952495813369751),
('gmisc' , 0.7929254770278931),
('standard-error' , 0.7914502620697021),
('kernel-density' , 0.791408360004425),
('ggvis' , 0.7911226153373718),


In [ ]:
import time
start = time.time()
WORD_RE = re.compile(r"\w+")
from pyspark.mllib.feature import Word2Vec
doc = valid_stack_posts_lines.map(lambda line: WORD_RE.findall(line))
model = Word2Vec().setVectorSize(100).setSeed(42).fit(doc)

syms = model.findSynonyms("ggplot2", 25)
end = time.time()
print((end - start)/60)

In [41]:
[s[0] for s in syms]

[]